<a href="https://colab.research.google.com/github/chloesung/Econometrics/blob/main/2.%20Instrument%20Variables%20Approach/Wooldridge/Wooldridge_Ch15(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
pip install wooldridge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install linearmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
import re
import math

import seaborn as sns
sns.set_palette("pastel")
sns.set(style="whitegrid")
import warnings
import matplotlib.pyplot as plt
pal = sns.color_palette('Set1')

from scipy.stats import t
from statsmodels.formula.api import ols
from statsmodels.api import add_constant
pd.options.display.float_format = '{:.5f}'.format
plt.rcParams['axes.unicode_minus'] = False
from linearmodels import IV2SLS
import wooldridge

warnings.filterwarnings(action='ignore')
get_ipython().run_line_magic('config', "InlineBackend.figure_format='retina' #화질 좋게 해주기")

# 1. College Proximity

## a) EDA

In [4]:
# Data Import
df = wooldridge.data('wage2')[['lwage', 'educ', 'exper', 'tenure', 'married', 'south', 'urban', 'black', 'IQ', 'KWW']]
df

,lwage,educ,exper,tenure,married,south,urban,black,IQ,KWW
0,6.64509,12,11,2,1,0,1,0,93,35
1,6.69456,18,11,16,1,0,1,0,119,41
2,6.71538,14,11,9,1,0,1,0,108,46
3,6.47697,12,13,7,1,0,1,0,96,32
4,6.33150,11,14,5,1,0,1,0,74,27
...,...,...,...,...,...,...,...,...,...,...
930,6.25383,16,6,1,1,1,0,1,79,28
931,7.09174,13,10,3,1,1,1,0,102,32
932,6.28786,12,12,10,1,1,0,1,77,22
933,6.77194,12,12,12,1,1,0,0,109,25


- *KWW* is an instrument for *IQ*

## b) OLS


In [5]:
# Regression
res = ols('lwage ~ educ + exper + tenure + married + south + urban + black + IQ', data = df).fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                       0.263
Model:                            OLS   Adj. R-squared:                  0.256
Method:                 Least Squares   F-statistic:                     41.27
Date:                Wed, 24 Aug 2022   Prob (F-statistic):           1.52e-56
Time:                        14:33:44   Log-Likelihood:                -375.09
No. Observations:                 935   AIC:                             768.2
Df Residuals:                     926   BIC:                             811.7
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.1764      0.128     40.441      0.0

- $\beta_1$ implies an almost 5.4% return for another year of education.

## c) IV

In [6]:
df = add_constant(df, has_constant="add")
dep = df['lwage']
exog = df[['const','educ', 'exper', 'tenure', 'married', 'south', 'urban', 'black']]
endog = df['IQ']
instr = df['KWW']

resultIV = IV2SLS(dep, exog, endog, instr).fit() 

print(resultIV.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.1900
Estimator:                    IV-2SLS   Adj. R-squared:                 0.1830
No. Observations:                 935   F-statistic:                    356.33
Date:                Wed, Aug 24 2022   P-value (F-stat)                0.0000
Time:                        14:33:44   Distribution:                  chi2(8)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          4.5925     0.3501     13.117     0.0000      3.9063      5.2786
educ           0.0250     0.0187     1.3410     0.17

- The coefficient on $educ$ is .025 ($se = .018.$)
- This is a low estimate, and it is not statistically different from zero.
- This is a puzzling finding, and it suggests that one of our assumptions fails; perhaps $e_1$ and $e_2$ are correlated.